In [2]:
from datasets import load_dataset

In [3]:
raw_dataset = load_dataset("khondoker/SentNoB")
raw_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/khondoker--SentNoB-1d4e2e75659b8316/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)
/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)
/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Data', 'Label'],
        num_rows: 15728
    })
})

In [4]:
size_in_mb = raw_dataset["train"].dataset_size
size_in_mb = size_in_mb/(1024**2)
print(f"Dataset size (cache file):{size_in_mb:.2f} MB")

Dataset size (cache file):3.31 MB


In [5]:
import psutil
print(f"RAM used:{psutil.Process().memory_info().rss/(1024*1024):.2f} MB")

RAM used:266.96 MB


In [6]:
ram_used = psutil.Process().memory_info().rss
ram_used = ram_used/(1024*1024)
print(f"RAM used:{ram_used:.2f} MB")


RAM used:266.96 MB


In [7]:
import timeit
code_snippet = """ 
batch_size = 1000
for idx in range(0, len(raw_dataset), batch_size):
    _ = raw_dataset["train"][idx : idx + batch_size]"""
time = timeit.timeit(stmt = code_snippet, number =1, globals = globals())

In [8]:
print(len(raw_dataset['train']), "examples")
print(f"size:{size_in_mb:.2f} MB")  
print(f"executed in: {time: .4f} seconds")
print(f"approximately {size_in_mb/time : .2f} MB/second")

15728 examples
size:3.31 MB
executed in:  0.0077 seconds
approximately  426.55 MB/second


In [9]:
large_dataset_streamed = load_dataset("EleutherAI/pile", streaming = True)
next(iter(large_dataset_streamed['train']))

{'text': 'It is done, and submitted. You can play “Survival of the Tastiest” on Android, and on the web. Playing on the web works, but you have to simulate multi-touch for table moving and that can be a bit confusing.\n\nThere’s a lot I’d like to talk about. I’ll go through every topic, insted of making the typical what went right/wrong list.\n\nConcept\n\nWorking over the theme was probably one of the hardest tasks I had to face.\n\nOriginally, I had an idea of what kind of game I wanted to develop, gameplay wise – something with lots of enemies/actors, simple graphics, maybe set in space, controlled from a top-down view. I was confident I could fit any theme around it.\n\nIn the end, the problem with a theme like “Evolution” in a game is that evolution is unassisted. It happens through several seemingly random mutations over time, with the most apt permutation surviving. This genetic car simulator is, in my opinion, a great example of actual evolution of a species facing a challenge.

In [10]:
large_dataset_streamed

{'train': <datasets.iterable_dataset.IterableDataset at 0x7cd57f702530>,
 'validation': <datasets.iterable_dataset.IterableDataset at 0x7cd57f702740>,
 'test': <datasets.iterable_dataset.IterableDataset at 0x7cd57f7029b0>}

In [11]:
large_dataset_streamed["train"]


In [12]:
type(large_dataset_streamed)

datasets.dataset_dict.IterableDatasetDict

In [13]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [14]:
tokenized_dataset = large_dataset_streamed['train'].map(lambda x : tokenizer(x['text']))
next(iter(tokenized_dataset))



Token indices sequence length is longer than the specified maximum sequence length for this model (3008 > 512). Running this sequence through the model will result in indexing errors


{'text': 'It is done, and submitted. You can play “Survival of the Tastiest” on Android, and on the web. Playing on the web works, but you have to simulate multi-touch for table moving and that can be a bit confusing.\n\nThere’s a lot I’d like to talk about. I’ll go through every topic, insted of making the typical what went right/wrong list.\n\nConcept\n\nWorking over the theme was probably one of the hardest tasks I had to face.\n\nOriginally, I had an idea of what kind of game I wanted to develop, gameplay wise – something with lots of enemies/actors, simple graphics, maybe set in space, controlled from a top-down view. I was confident I could fit any theme around it.\n\nIn the end, the problem with a theme like “Evolution” in a game is that evolution is unassisted. It happens through several seemingly random mutations over time, with the most apt permutation surviving. This genetic car simulator is, in my opinion, a great example of actual evolution of a species facing a challenge.

In [15]:
dataset_head = large_dataset_streamed['train'].take(5)
list(dataset_head)

[{'text': 'It is done, and submitted. You can play “Survival of the Tastiest” on Android, and on the web. Playing on the web works, but you have to simulate multi-touch for table moving and that can be a bit confusing.\n\nThere’s a lot I’d like to talk about. I’ll go through every topic, insted of making the typical what went right/wrong list.\n\nConcept\n\nWorking over the theme was probably one of the hardest tasks I had to face.\n\nOriginally, I had an idea of what kind of game I wanted to develop, gameplay wise – something with lots of enemies/actors, simple graphics, maybe set in space, controlled from a top-down view. I was confident I could fit any theme around it.\n\nIn the end, the problem with a theme like “Evolution” in a game is that evolution is unassisted. It happens through several seemingly random mutations over time, with the most apt permutation surviving. This genetic car simulator is, in my opinion, a great example of actual evolution of a species facing a challenge

In [16]:
train_dataset = large_dataset_streamed["train"].skip(1000)
validation_dataset = large_dataset_streamed["train"].take(1000)

In [17]:
#list(validation_dataset)

In [18]:
ram_used = psutil.Process().memory_info().rss
ram_used = ram_used/(1024*1024)
print(f"RAM used:{ram_used:.2f} MB")


RAM used:344.97 MB
